In [8]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from openai import AzureOpenAI 
from dotenv import load_dotenv
import os


In [9]:

load_dotenv()

# ✅ Step 3: Fetch environment variables from the system
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    api_version=azure_api_version
)


In [10]:

class BlogState(TypedDict):
    title:str
    outline:str
    content:str


In [11]:
# ✅ Function 1: Create an outline for the blog topic
def create_outline(state: BlogState) -> BlogState:
    """
    Generates a blog outline based on the given title using Azure OpenAI.
    Updates the state with the generated outline.
    """
    title = state['title']
    prompt = f"Generate a detailed outline for a blog post on the topic: '{title}'."

    try:
        # Azure OpenAI chat completion request
        response = client.chat.completions.create(
            model=azure_deployment,  # Azure deployment name from your environment
            messages=[
                {"role": "system", "content": "You are a skilled blog writer and content strategist."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,  # Controls creativity
            max_tokens=400    # Increased for more detailed outlines
        )

        # Extract model's response text
        outline = response.choices[0].message.content.strip()
        state['outline'] = outline

    except Exception as e:
        # Store error message in the state if API call fails
        state['outline'] = f"Error while generating outline: {str(e)}"

    return state


# ✅ Function 2: Create the actual blog content from the outline
def create_blog(state: BlogState) -> BlogState:
    """
    Generates a 300-word blog post based on the previously created outline.
    Updates the state with the generated content.
    """
    outline = state.get('outline', '')

    # If outline is empty or contains an error, handle gracefully
    if not outline or outline.startswith("Error"):
        state['content'] = "Cannot generate blog content because outline is missing or invalid."
        return state

    prompt = f"Write a short, engaging blog post of about 300 words based on this outline:\n\n{outline}"

    try:
        # Azure OpenAI chat completion request
        response = client.chat.completions.create(
            model=azure_deployment,  # Azure deployment name
            messages=[
                {"role": "system", "content": "You are a creative content writer who writes in a clear and engaging tone."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8,  # Slightly more creative for blog writing
            max_tokens=600    # Allow enough tokens for ~300 words
        )

        # Extract and clean up the response text
        content = response.choices[0].message.content.strip()
        state['content'] = content

    except Exception as e:
        # Store any error message in the state
        state['content'] = f"Error while generating content: {str(e)}"

    return state

In [14]:
graph=StateGraph(BlogState)

graph.add_node("create_outline",create_outline)
graph.add_node("create_blog",create_blog)

graph.add_edge(START,"create_outline")
graph.add_edge("create_outline","create_blog")

graph.add_edge("create_blog",END)

workflow=graph.compile()

output=workflow.invoke({"title":"mera bharat mahan"})
print(output)


{'title': 'mera bharat mahan', 'outline': '**Detailed Blog Outline: "Mera Bharat Mahan"**\n\n### **Introduction**\n1. **Engaging Hook**: Begin with a powerful and emotional quote or statement about India’s greatness.\n   - Example: "India’s essence lies in its diversity, unity, and resilience. Truly, \'Mera Bharat Mahan.\'"\n2. **Purpose of the Blog**: Explain why it’s important to celebrate and explore the greatness of India.\n3. **Brief Overview**: Highlight the themes that will be covered in the blog (culture, history, achievements, and values).\n\n---\n\n### **I. Rich Heritage and Cultural Diversity**\n1. **Diversity in Culture**  \n   - Discuss India’s linguistic diversity (22 official languages, 1600+ dialects).  \n   - Explore the vivid festivals celebrated across states (Diwali, Holi, Eid, Christmas, etc.).  \n   - Mention the variety of cuisines that represent different regions.  \n\n2. **Historical Significance**  \n   - Highlight India’s ancient civilizations (Indus Valley, 